In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os
import json
import pandas as pd
import numpy as np
import statistics


In [138]:
import re
import datetime

In [64]:
df = pd.read_csv('data/ownerdf.csv')
spydf = pd.read_csv('data/steamspy.csv')

In [14]:
df.head(3)

,title,owners_median,positive,negative,num_reviews,ccu,median_2weeks,median_forever,average_2weeks,average_forever,...,Electronic Arts,Paradox Interactive,SEGA,Sokpop Collective,Square Enix,THQ Nordic,Ubisoft,Winged Cloud,other,tinyBuild
0,OUTRIDERS,1500000.0,26922,14940,36837,23037,279,1600,585,2074,...,0,0,0,0,1,0,0,0,0,0
1,It Takes Two,750000.0,20205,877,15688,11513,425,753,371,670,...,1,0,0,0,0,0,0,0,0,0
2,Valheim,15000000.0,222551,8466,222855,64931,435,1113,820,2205,...,0,0,0,0,0,0,0,0,1,0


In [66]:
spydf.head(3)

,Unnamed: 0,appid,title,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,owners_median
0,240760,240760,Wasteland 2: Director's Cut,inXile Entertainment,inXile Entertainment,NaN,6499,1385,0,"500,000 .. 1,000,000",877,0,681,0,2999.0,2999.0,0.0,32,750000.0
1,241540,241540,State of Decay,Undead Labs,Xbox Game Studios,NaN,14963,2744,0,"500,000 .. 1,000,000",1304,0,1461,0,0.0,0.0,0.0,18,750000.0
2,8800,8800,Civilization IV: Beyond the Sword,Firaxis Games,2K,NaN,2694,100,0,"500,000 .. 1,000,000",3395,196,4337,196,999.0,999.0,0.0,1576,750000.0


In [15]:
df.shape

(5188, 92)

In [668]:
chromedriver = "/Applications/chromedriver" 
os.environ["webdriver.chrome.driver"] = chromedriver
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')

#url = 'https://steamspy.com/search.php'
#driver.get('steamspy.com/app/240760')

driver = webdriver.Chrome(chromedriver)
driver.get(url)



In [215]:
driver.get(url)

# Game Specific Scraping

In [68]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [77]:
#title of game
title = soup.find('h3').text
title

"Wasteland 2: Director's Cut"

In [69]:
#number of owners
num_owners = int(soup.find('strong',text='Owners').next_sibling.strip().split()[1].replace(',',''))
num_owners

804000

In [70]:
#scored by users
userscore = int(soup.find('strong', text = 'Old userscore:').next_sibling.strip().replace('%',''))
userscore

82

In [75]:
#scored by critics
metascore = int(soup.find('strong', text = 'Metascore:').next_sibling.strip().replace('%',''))
metascore

87

In [117]:
#finding list of languages from this more complete site
languages = [language.text for language in soup.find_all('a', href=re.compile('language'))]

#removes the abreviations that were causing doubles of certain languages
for items in languages:
    if len(items) <= 2:
        languages.remove(items)

In [118]:
languages

['English',
 'Russian',
 'German',
 'Spanish',
 'Chinese',
 'French',
 'Polish',
 'Turkish',
 'Swedish',
 'English',
 'French',
 'German',
 'Spanish - Spain',
 'Polish',
 'Russian',
 'Italian',
 'Turkish']

In [159]:
#average total playtime
avg_playtime = pd.to_datetime(soup.find('strong', text = 'Playtime total:').next_sibling.split()[0]).time()
avg_playtime

datetime.time(18, 57)

In [137]:
#number of users following the game
num_followers = int(soup.find('strong', text = 'Followers').next_sibling.split()[1].replace(',',''))
num_followers

94336

# Creating list of game titles to scrape from second site

pulling game id's from spydf for all titles in reviewsdf

In [160]:
spyscrapedf = pd.merge(reviewsdf['title'],spydf[['title','appid']],on='title')

In [161]:
spyscrapedf


,title,appid
0,OUTRIDERS,680420
1,It Takes Two,1426210
2,Valheim,892970
3,7 Days to Die,251570
4,Battlefield V,1238810
...,...,...
5184,The Prism,399420
5185,Blaster Simulator,446450
5186,Rail Cargo Simulator,446490
5187,Redemption: Eternal Quest,390880


In [271]:
len(spydf)

42773

In [180]:
idlist = list(spyscrapedf.appid)

In [202]:
urllist=[]
for items in idlist:
    urllist.append('https://steamspy.com/app/'+ str(items))

In [203]:
urllist[:4]

['https://steamspy.com/app/680420',
 'https://steamspy.com/app/1426210',
 'https://steamspy.com/app/892970',
 'https://steamspy.com/app/251570']

In [266]:
def get_spy_details(url):
    
    
    '''
    Similar to get_game_details, this function will run though the urls it is passed, and scrape the appropirate information 
    for each game from steamspy. Details being scraped from this site are: title, number of owners, userscore, metascore (critic),
    languages (seems to be more easily attained on this site than steam itself), average total playtime, and number of people
    following the game. Will give nans if details are not found on the page
    '''
    #url = 'https://steamspy.com/app/{}'.format(item)
    response = driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    
    headers = ['title','owners','userscore',
              'metascore','languages','avg_playtime',
              'num_followers']
    
    
    #get the title
    if not soup.find('h3').text:
        title = np.nan
    else:
        title = title = soup.find('h3').text
 

    #get the number of owners
    if not soup.find('strong',text='Owners').next_sibling:
        owners = np.nan
    else:
        owners = int(soup.find('strong',text='Owners').next_sibling.strip().split()[1].replace(',',''))
    
    
    #get the userscore
    if not soup.find('strong', text = 'Old userscore:'):
        userscore = np.nan
    else:
        userscore = int(soup.find('strong', text = 'Old userscore:').next_sibling.strip().replace('%',''))
        
        
    #get the metacritic score
    if not soup.find('strong', text = 'Metascore:'):
        metascore = np.nan
    else:
        metascore = int(soup.find('strong', text = 'Metascore:').next_sibling.strip().replace('%',''))
        
        
    #get languages
    if not soup.find_all('a', href=re.compile('language')):
        languages = np.nan
    else:
        languages = [language.text for language in soup.find_all('a', href=re.compile('language'))]
        #removes the abreviations that were causing doubles of certain languages
        for items in languages:
            if len(items) <= 2:
                languages.remove(items)
                
                
    #get average playtime
    if not soup.find('strong', text = 'Playtime total:'):
        avg_playtime = np.nan
    else:
        #avg_playtime = pd.to_datetime(soup.find('strong', text = 'Playtime total:').next_sibling.split()[0]).time()
        avg_playtime = soup.find('strong', text = 'Playtime total:').next_sibling.split()[0]
        
        
        
    #get number of followers
    if not soup.find('strong', text = 'Followers'):
        followers = np.nan
    elif not soup.find('strong', text = 'Followers').next_sibling:
        followers = np.nan
    else:
        followers = int(soup.find('strong', text = 'Followers').next_sibling.split()[1].replace(',',''))

        
    spydict = dict(zip(headers,[title, owners, userscore,
                               metascore, languages, avg_playtime,
                               followers]))
    
    return spydict

In [210]:
spy_info = []

for url in urllist:
    spy_info.append(get_spy_details(url))

AttributeError: 'NoneType' object has no attribute 'next_sibling'

In [268]:
len(spy_info)

5189

In [269]:
len(urllist)

5189

In [222]:
chrome_options.add_argument('--headless')

In [267]:
for url in urllist[2462:]:
    spy_info.append(get_spy_details(url))

# EDA on data screaped on SteamSpy

Transforming scraped data into a dataframe and exploring the results

In [272]:
spyscrapedf  = pd.DataFrame(spy_info)

In [585]:
spyscrapedf

,title,owners,userscore,metascore,languages,avg_playtime,num_followers
0,OUTRIDERS,1811000,64.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",53:52,121322.0
1,It Takes Two,776000,95.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",12:49,99446.0
2,Valheim,15510000,96.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",35:21,333665.0
3,7 Days to Die,7988000,86.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",66:24,403936.0
4,Battlefield V,1037000,66.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",25:10,67662.0
...,...,...,...,...,...,...,...
5238,The Prism,49000,73.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",05:32,262.0
5239,Blaster Simulator,9000,NaN,NaN,"[English, Russian, German, Spanish, Chinese, F...",NaN,185.0
5240,Rail Cargo Simulator,5000,NaN,NaN,"[English, Russian, German, Spanish, Chinese, F...",00:06,141.0
5241,Redemption: Eternal Quest,126000,47.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",03:35,661.0


In [586]:
spyscrapedf.describe()

,owners,userscore,metascore,num_followers
count,5.168000e+03,4175.000000,1199.000000,5.167000e+03
mean,2.985223e+05,82.719521,75.842369,2.248680e+04
std,1.251051e+06,12.856312,9.426422,1.317565e+05
min,0.000000e+00,13.000000,23.000000,0.000000e+00
25%,5.000000e+03,77.000000,71.000000,3.680000e+02
50%,3.000000e+04,86.000000,77.000000,2.583000e+03
75%,1.732500e+05,92.000000,82.000000,1.311700e+04
max,5.550900e+07,100.000000,96.000000,7.597045e+06


In [277]:
spyscrapedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5189 entries, 0 to 5188
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5189 non-null   object 
 1   owners         5189 non-null   int64  
 2   userscore      4178 non-null   float64
 3   metascore      1202 non-null   float64
 4   languages      5189 non-null   object 
 5   avg_playtime   3012 non-null   object 
 6   num_followers  5188 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 283.9+ KB


In [588]:
len(spyscrapedf.loc[0,'languages'])

22

In [589]:
spyscrapedf['num_languages'] = len()

# Number of Languages
more complete list of languages from SteamSpy

In [611]:
for i in range(len(spyscrapedf)):
    spyscrapedf.loc[i,'num_languages'] = len(spyscrapedf.loc[i,'languages'])

In [613]:
spyscrapedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5168 entries, 0 to 5167
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5168 non-null   object 
 1   owners         5168 non-null   int64  
 2   userscore      4175 non-null   float64
 3   metascore      1199 non-null   float64
 4   languages      5168 non-null   object 
 5   avg_playtime   3009 non-null   object 
 6   num_followers  5167 non-null   float64
 7   num_languages  5168 non-null   int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 323.1+ KB


# Userscore

there are only 1k missing a userscore, going to fill with 0, as no score is recorded

In [618]:
spyscrapedf.userscore = spyscrapedf.userscore.fillna(0)

In [619]:
spyscrapedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5168 entries, 0 to 5167
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5168 non-null   object 
 1   owners         5168 non-null   int64  
 2   userscore      5168 non-null   float64
 3   metascore      1199 non-null   float64
 4   languages      5168 non-null   object 
 5   avg_playtime   3009 non-null   object 
 6   num_followers  5167 non-null   float64
 7   num_languages  5168 non-null   int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 323.1+ KB


# MetaScore
These are games that have a metacritic score. Only some have this value, so going to make it binary. Hopefully will highlight if this makes a game more popular or not

In [621]:
spyscrapedf['has_meta']=0

In [632]:


spyscrapedf.loc[spyscrapedf.metascore>0,['has_meta']]=1


In [633]:
spyscrapedf.has_meta.value_counts()

0    3969
1    1199
Name: has_meta, dtype: int64

In [634]:
spyscrapedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5168 entries, 0 to 5167
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5168 non-null   object 
 1   owners         5168 non-null   int64  
 2   userscore      5168 non-null   float64
 3   metascore      1199 non-null   float64
 4   languages      5168 non-null   object 
 5   avg_playtime   3009 non-null   object 
 6   num_followers  5167 non-null   float64
 7   num_languages  5168 non-null   int64  
 8   has_meta       5168 non-null   int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 363.5+ KB


# Number of Followers
One game has no number of followers, filling in with 0

In [635]:
spyscrapedf[spyscrapedf.num_followers.isna()]

,title,owners,userscore,metascore,languages,avg_playtime,num_followers,num_languages,has_meta
2455,Star Control III,0,0.0,89.0,"[English, Russian, German, Spanish, Chinese, F...",NaN,NaN,10,1


In [666]:
spyscrapedf.num_followers = spyscrapedf.num_followers.fillna(0)

# Saving DataFrame
as spyscrapedf

In [667]:
spyscrapedf.to_csv(r'/Users/MichaelHarnett/Desktop/METIS/Steam_Analysis/Data\spyscrapedf.csv',index=False)

# Missing owners
scraping the missing 234 games again for ownership info

In [636]:
spyscrapedf[spyscrapedf.owners==0]

,title,owners,userscore,metascore,languages,avg_playtime,num_followers,num_languages,has_meta
59,Garry's Mod,0,96.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",172:46,0.0,34,0
180,Portal,0,98.0,90.0,"[English, Russian, German, Spanish, Chinese, F...",05:35,0.0,26,1
324,"Star Wars: Battlefront 2 (Classic, 2005)",0,93.0,78.0,"[English, Russian, German, Spanish, Chinese, F...",11:18,74777.0,10,1
340,Counter-Strike: Source,0,96.0,88.0,"[English, Russian, German, Spanish, Chinese, F...",154:35,0.0,21,1
397,Total War: SHOGUN 2,0,90.0,90.0,"[English, Russian, German, Spanish, Chinese, F...",41:11,124974.0,17,1
...,...,...,...,...,...,...,...,...,...
4601,Warrior Kings: Battles,0,93.0,64.0,"[English, Russian, German, Spanish, Chinese, F...",NaN,675.0,10,1
4715,Gunship!,0,44.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",NaN,341.0,10,0
4988,Rag Doll Kung Fu,0,71.0,69.0,"[English, Russian, German, Spanish, Chinese, F...",01:46,249.0,10,1
5091,Bad Day,0,64.0,NaN,"[English, Russian, German, Spanish, Chinese, F...",NaN,63.0,10,0


In [638]:
missing_titles = spyscrapedf[spyscrapedf.owners==0]

In [639]:
missing_titles_id = pd.merge(missing_titles[['title']],spydf[['title','appid']],on='title')

In [643]:
missing_titles_id.drop_duplicates(subset='title',inplace=True)

In [644]:
missing_titles_id

,title,appid
0,Garry's Mod,4000
1,Portal,400
2,"Star Wars: Battlefront 2 (Classic, 2005)",6060
3,Counter-Strike: Source,240
4,Total War: SHOGUN 2,34330
...,...,...
230,Warrior Kings: Battles,299070
231,Gunship!,286730
232,Rag Doll Kung Fu,1002
233,Bad Day,803080


In [656]:
missing_id_list = list(missing_titles_id.appid)
missing_id_list[:3]

missing_url_list=[]
for items in missing_id_list:
    missing_url_list.append('https://steamspy.com/app/'+ str(items))

In [658]:
len(missing_url_list)

234

In [661]:
def just_owners(url):
    
    
    response = driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    
    headers = ['title','owners']
    
    
    #get the title
    if not soup.find('h3').text:
        title = np.nan
    else:
        title = title = soup.find('h3').text
 

    #get the number of owners
    if not soup.find('strong',text='Owners').next_sibling:
        owners = np.nan
    else:
        owners = int(soup.find('strong',text='Owners').next_sibling.strip().split()[1].replace(',',''))
        
    spydict = dict(zip(headers,[title, owners]))
    
    return spydict    
    
    

In [669]:
missing_info = []


for url in missing_url_list[21:]:
    missing_info.append(just_owners(url))

In [670]:
len(missing_info)

213

In [672]:
missingdf = pd.DataFrame(missing_info)

In [675]:
missingdf.tail(10)

,title,owners
203,Alien Shooter,200000
204,Paws and Claws: Pet Vet,0
205,Miss Popularity,0
206,Paws and Claws: Pet School,0
207,Warrior Kings,20000
208,Warrior Kings: Battles,20000
209,Gunship!,0
210,Rag Doll Kung Fu,0
211,Bad Day,20000
212,HexON,0


## Michael
ask about your personalized dummy columns! specficially, when using pd.get_dummies, you drop the first column, which amkes 
sesne when they explain the absence of all values or if every oteher dummy was 0, then the last one is chosen,  but we did not 
account for this with our creation. 
actualy maybe 'other' caccounts for this? yea I will still follow up, but im assuming that by includig an other term, its the same as dropping hte first dummy colum

either way, if i werer to use get_dummies twice or more on the same dataset, how does this help/affect data?

In [261]:
drver.quit()

NameError: name 'drver' is not defined

In [484]:
driver.quit()